In [ ]:
%matplotlib inline
import utils; reload(utils)
from utils import *

In [ ]:
# path = '..data/redux/sample/'
path = '../data/redux/'
model_path = path + 'models/'

In [ ]:
# batch_size = 4
batch_size = 64

In [ ]:
model = vgg_ft(2)

In [ ]:
model.summary()

In [ ]:
# import previously fine tuned model's weights
model.load_weights(path+'results/ft1.h5')

In [ ]:
layers = model.layers

In [ ]:
conv_layers_idxs = [i for i,layer in enumerate(layers) if type(layer) == Convolution2D]

In [ ]:
last_conv_layer_idx = conv_layers_idxs[-1]
last_conv_layer_idx

In [ ]:
conv_layers = layers[:last_conv_layer_idx+1]
conv_model = Sequential(conv_layers)

In [ ]:
fc_layers = layers[last_conv_layer_idx+1:]

In [ ]:
batches = get_batches(path+'train/',batch_size=batch_size)
val_batches = get_batches(path+'valid/',batch_size=batch_size)

In [ ]:
trn_classes = batches.classes
valid_classes = val_batches.classes

In [ ]:
trn_labels = onehot(trn_classes)
valid_labels = onehot(valid_classes)

In [ ]:
val_batches.nb_sample

In [ ]:
# trn_features = conv_model.predict_generator(batches,batches.nb_sample)
# val_features = conv_model.predict_generator(val_batches,val_batches.nb_sample)

In [ ]:
# save_array(model_path+'conv_trn_features.bc',trn_features)
# save_array(model_path+'conv_val_features.bc',val_features)

In [ ]:
trn_features = load_array(model_path+'conv_trn_features.bc')
val_features = load_array(model_path+'conv_val_features.bc')

In [ ]:
trn_features.shape

In [ ]:
def proc_wgts(layer) :return [o/2 for o in layer.get_weights()]

In [ ]:
from keras.optimizers import SGD, RMSprop, Adam
opt = RMSprop(lr=0.00001, rho=0.7)

In [ ]:
def fc_model():
    model = Sequential([
            MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
            Flatten(),
            Dense(4096,activation='relu'),
            Dropout(0.),
            Dense(4096,activation='relu'),
            Dropout(0.),
            Dense(2,activation='softmax')
        ])
    for l1,l2 in zip(model.layers,fc_layers):
        l1.set_weights(proc_wgts(l2))
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
fc_model = fc_model()

In [ ]:
fc_model.fit(trn_features,trn_labels,
            nb_epoch=8,batch_size=batch_size,
            validation_data=(val_features,valid_labels))

In [ ]:
fc_model.summary()

In [ ]:
fc_model.save_weights(model_path+'no_dropout.h5')

In [ ]:
fc_model.load_weights(model_path+'no_dropout.h5')

#### Approaches to Reduce Overfitting

1. Add more data
2. Data Augmentation
3. Reduce architecture complexity or use architectures that generalise well.
4. Add Regularisation or Dropouts

#### Data Augmentation

In [ ]:
from keras.preprocessing import image
import numpy as np
gen = image.ImageDataGenerator(rotation_range=10,
                               width_shift_range=0.1,
                               height_shift_range=0.1,
                               horizontal_flip=True,
                               zoom_range=0.1
                              )

In [ ]:
from scipy import ndimage
# img_loc = 'data/dogscats/test/7.jpg'
img = np.expand_dims(ndimage.imread('..data/dogscats/test/7.jpg'),0)

In [ ]:
img.shape

In [ ]:
aug_iter = gen.flow(img)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(img[0])
plt.show()

In [ ]:
aug_imgs = [next(aug_iter)[0].astype(np.uint8) for i in range(8)]

In [ ]:
from utils import *
plots(aug_imgs,(20,7),2)
plt.show()

In [ ]:
# ??np.expand_dims
K.set_image_dim_ordering('th')

#### Augmenting Data

In [ ]:
gen = image.ImageDataGenerator(rotation_range=15,
                               width_shift_range=0.1,
                               height_shift_range=0.1,
                               horizontal_flip=True,
                               zoom_range=0.1
                        )

In [ ]:
batches = get_batches(path+'train/',gen,batch_size=batch_size)
val_batches = get_batches(path+'valid/',batch_size=batch_size)

In [ ]:
for layer in conv_model.layers: layer.trainable = False

In [ ]:
conv_model.add(fc_model)

In [ ]:
conv_model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
conv_model.fit_generator(batches,nb_epoch=8,validation_data=val_batches)

In [ ]:
conv_model.save_weights(model_path+'aug1.h5')

In [ ]:
conv_model.load_weights(model_path+'aug1.h5')